## **Agents**

An agent-based workflow where LLMs act autonomously within a loop, interacting with their environment and receiving feedback to refine their actions and decisions.

<img src="https://raw.githubusercontent.com/BrightPool/udemy-prompt-engineering-course/refs/heads/main/images/autonomous-agent.webp" alt="Agents" width="500">

### Learning Goals
- Understand how tool-calling works in a simple agent loop.
- See how assistant tool-calls must be followed by `role: tool` responses.
- Build a minimal loop that runs until no more tool-calls are required.

### How It Works
1. User asks a question.
2. Model may request tools (functions) by emitting `tool_calls`.
3. Your code executes the requested tools and replies with `role: tool` messages using each `tool_call_id`.
4. Repeat until the model stops requesting tools; then show the final answer.

## **Use Cases:**

- Building a personal research assistant that autonomously searches academic papers, extracts key findings, and generates literature review summaries based on specific research questions.
- Creating an autonomous code reviewer that analyzes pull requests, identifies potential bugs and security issues, suggests improvements, and generates detailed review comments.
- Developing a customer support agent that handles inquiries by searching knowledge bases, generating appropriate responses, and escalating complex issues to human agents when needed.
- Managing social media presence by analyzing trending topics, generating relevant content, scheduling posts, and engaging with followers through personalized responses.
- Building an autonomous testing agent that generates test cases, executes tests, analyzes failures, and provides detailed bug reports with suggested fixes.
- Creating a data monitoring agent that continuously analyzes system metrics, detects anomalies, investigates root causes, and generates incident reports with recommended actions.

### Lesson Outline
- Define simple tools and a single tools schema.
- Use a minimal while-loop to process tool-calls correctly.
- Print the final answer once the model stops requesting tools.

Tip: You can change the user message (e.g., ask about cryptography or materials discovery) and re-run the loop to see different tool behavior.

In [1]:
# %pip install openai pydantic --upgrade

In [37]:
import json
import logging
from openai import OpenAI

# Simple logging setup for the notebook
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s [agent] %(message)s"
)

# Helper to toggle log level at runtime
_LEVELS = {
    "CRITICAL": logging.CRITICAL,
    "ERROR": logging.ERROR,
    "WARNING": logging.WARNING,
    "INFO": logging.INFO,
    "DEBUG": logging.DEBUG,
}

def set_log_level(level_name: str = "INFO") -> None:
    level = _LEVELS.get(level_name.upper(), logging.INFO)
    logging.getLogger().setLevel(level)
    logging.info(f"Log level set to {level_name.upper()}")


### Logging Level Toggle
Use the helper to switch verbosity during the exercise:
- Example: set INFO (default) or DEBUG for more detail.
- Call `set_log_level("DEBUG")` before running the agent loop to see more logs.

In [38]:
client = OpenAI()
MODEL="gpt-4o-mini"

In [41]:
# Sample tools the agent can call
# In practice, connect real APIs/services.

def search_knowledge_base(query: str) -> str:
    """
    Return a short, placeholder summary for the given query.
    """
    return (
        f"Summary for '{query}': Quantum computing uses qubits to leverage "
        f"superposition and entanglement for certain problems beyond classical approaches."
    )


def generate_keywords(query: str) -> str:
    """
    Return a simple comma-separated list of keywords for the query.
    """
    return "Qubits, Superposition, Entanglement"

# Function tool schema for the model
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_knowledge_base",
            "description": "Query a knowledge base to retrieve relevant info on a topic.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The user question or search query."}
                },
                "required": ["query"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    },
    {
        "type": "function",
        "function": {
            "name": "generate_keywords",
            "description": "Generate relevant keywords about a topic.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The user question or search query."}
                },
                "required": ["query"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    },
]

# Simple dispatcher so we avoid long if/elif chains
TOOL_DISPATCH = {
    "search_knowledge_base": search_knowledge_base,
    "generate_keywords": generate_keywords,
}

# Initial user message
messages = [{"role": "user", 
            "content": "Hello, I'd like to know more about quantum computing."}]


### Try It: Change the Prompt
Reset the user prompt and rebuild the conversation to explore different topics. Then run the agent loop.
- Example topics: "basic cryptography", "materials discovery", "quantum error correction".
- Edit `user_prompt` below and run the next cell.

In [34]:
# Set a new topic and rebuild the conversation
user_prompt = "Give me a basic cryptography overview."
messages = [{"role": "user", "content": user_prompt}]

# Optional: increase logging verbosity for this run
# set_log_level("DEBUG")


### Agent Loop Explained
This loop lets the model request tools, executes them, and returns the results until no more tools are needed.

- Start: The loop runs with the current `messages` (your conversation so far).
- Call model: We ask the model for a response; it may include `tool_calls`.
- Check `tool_calls`: If none are returned, the loop ends and we show the final answer.
- Append assistant message: If there are tool calls, we append the assistant’s message (which contains them) to `messages`.
- Execute tools: For each tool call, we run the matching Python function with the provided JSON arguments.
- Reply with tool outputs: For every tool call, we append a `role: tool` message using the exact `tool_call_id`.
- Repeat: With the new tool outputs in `messages`, we call the model again. This continues until no tool calls are requested.

### Tool Execution in the Loop (Cell 13)
- Detect and collect tool calls: The model returns a list of `tool_calls`, each with `function.name`, `function.arguments` (JSON), and a unique `id`.
- Append assistant tool-call message: We add the assistant message (containing the `tool_calls`) to `messages` before executing any tools.
- Execute each tool in order: For every call, we parse args via `json.loads(call.function.arguments)` and dispatch to `search_knowledge_base()` or `generate_keywords()` based on `call.function.name`.
- Collect results per call: We build `{"tool_call_id": call.id, "content": result}` for each executed tool.
- Respond to the model per call: We append a `role: tool` message for each result using its exact `tool_call_id`. This is required by the API.
- All tools are executed: If multiple tools are requested in a single iteration, we execute all of them (none are skipped), then call the model again.
- Ordering: Tools are executed sequentially in the order provided by `tool_calls` (not in parallel).

Why the `tool_call_id` matters:
- Each assistant tool call must be followed by a `role: tool` message for that specific `tool_call_id`, otherwise the API returns an error.

What to look for in logs:
- Iteration number and how many `tool_calls` were returned (with tool names).
- Tool names and arguments when they are executed.
- Confirmation that tool responses were appended before the next model call.

### Pseudo-Flow
- messages → call model → tool_calls?
  - yes → append assistant(tool_calls) → for each call: parse args → execute tool → append role: tool(tool_call_id, content) → repeat
  - no  → final assistant answer (print)


In [40]:
# Minimal Agent Loop: executes requested tools until none remain
# Uses the `messages` list defined above.

iteration = 0
while True:
    # 1) Ask the model what to do next
    logging.info(f"Iteration {iteration}: calling model with {len(messages)} messages")
    completion = client.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    tool_calls = completion.choices[0].message.tool_calls

    count = len(tool_calls) if tool_calls else 0
    if tool_calls:
        names = ", ".join([call.function.name for call in tool_calls])
        logging.info(f"Iteration {iteration}: tool_calls returned = {count} ({names})")
    else:
        logging.info(f"Iteration {iteration}: tool_calls returned = 0")

    # 2) If no tool calls, we are done
    if not tool_calls:
        logging.info(f"Iteration {iteration}: no tool calls; exiting loop")
        break

    # 3) Add the assistant message that requested the tools
    logging.info(f"Iteration {iteration}: appending assistant message with tool_calls [{names}]")
    messages.append(completion.choices[0].message)

    # 4) Execute each requested tool and prepare responses
    results = []
    for call in tool_calls:
        function_name = call.function.name
        function_args = json.loads(call.function.arguments)
        handler = TOOL_DISPATCH.get(function_name)
        logging.info(f"Iteration {iteration}: executing tool '{function_name}' with args {function_args}")

        if handler:
            result = handler(**function_args)
        else:
            logging.warning(f"Iteration {iteration}: unknown tool '{function_name}'")
            result = "Error: Unknown function"

        results.append({"tool_call_id": call.id, "content": result, "name": function_name})

    # 5) Send tool responses back to the model, matching tool_call_id
    for r, call in zip(results, tool_calls):
        messages.append({"role": "tool", "tool_call_id": r["tool_call_id"], "content": r["content"]})
        logging.info(
            f"Iteration {iteration}: appended tool response for {call.function.name} "
            f"(tool_call_id={r['tool_call_id']})"
        )

    logging.info(f"Iteration {iteration}: appended {len(results)} total tool responses")
    iteration += 1


2025-12-23 17:14:02,215 INFO [agent] Iteration 0: calling model with 1 messages
2025-12-23 17:14:04,231 INFO [agent] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-23 17:14:04,234 INFO [agent] Iteration 0: tool_calls returned = 2 (search_knowledge_base, generate_keywords)
2025-12-23 17:14:04,234 INFO [agent] Iteration 0: appending assistant message with tool_calls [search_knowledge_base, generate_keywords]
2025-12-23 17:14:04,235 INFO [agent] Iteration 0: executing tool 'search_knowledge_base' with args {'query': 'What is quantum computing?'}
2025-12-23 17:14:04,236 INFO [agent] Iteration 0: executing tool 'generate_keywords' with args {'query': 'quantum computing'}
2025-12-23 17:14:04,236 INFO [agent] Iteration 0: appended tool response for search_knowledge_base (tool_call_id=call_z0Xp3mEurzB5jjkEd6rMCSLC)
2025-12-23 17:14:04,236 INFO [agent] Iteration 0: appended tool response for generate_keywords (tool_call_id=call_9e2b41ChmMNID40u09i8vFCM)


In [36]:
# Show the final assistant message after the loop
final_answer = completion.choices[0].message.content
print("Final Answer from Agent:\n")
print(final_answer)

Final Answer from Agent:

It seems that the search results did not return relevant information specifically about basic cryptography. However, here's a basic overview of cryptography based on general knowledge:

### What is Cryptography?
Cryptography is the practice and study of techniques for securing communication and information from adversaries. It involves creating written or generated codes that allow information to be kept secret.

### Key Concepts:
1. **Encryption**: The process of converting plaintext (readable data) into ciphertext (encoded data) using algorithms and keys. Only those with the correct key can decrypt the ciphertext back into plaintext.
   
2. **Decryption**: The reverse process of encryption, transforming ciphertext back into plaintext.

3. **Keys**: Strings of bits used by encryption algorithms. The security of encryption relies on the length and complexity of the keys.

4. **Hash Functions**: Cryptographic algorithms that take an input and produce a fixed-si